In [16]:
%load_ext autoreload
%autoreload 2


In [17]:
import yt

from modules import *
from sim import *
from read_ramses import *


In [20]:
# choose simulation
sim_round = 3
sim_name = "alpha_eps1p0"
sim_dir = move_to_sim_dir(sim_round, sim_name)
print()

dump = get_dump(a_exp=0.1)
info = get_info(dump)

print("Dump: %d" % dump)
print("a_exp: %.3g" % info.a_exp)
print("Maximum AMR level: %d" % info.amr_level_max)


Moving to directory '/home/za9132/scratch/romain/round3/alpha_eps1p0'.

Dump: 41
a_exp: 0.1
Maximum AMR level: 13


In [21]:
use_cubic = False

halo_cat = get_halo_cat(dump, cgs=True)
biggest_halo_idx = np.argmax(halo_cat.mass)
biggest_halo_coord_cubic = get_biggest_halo_coord_cubic(info.a_exp) * info.length_unit

print("Halo coord (cubic): (%.5g, %.5g, %.5g)" % tuple(biggest_halo_coord_cubic / info.length_unit))
print("Halo coord (mass): (%.5g, %.5g, %.5g)" % tuple(halo_cat.coord[:, biggest_halo_idx] / info.length_unit))
if use_cubic: halo_cat.coord[:, biggest_halo_idx] = biggest_halo_coord_cubic


Halo coord (cubic): (0.51053, 0.51273, 0.49297)
Halo coord (mass): (0.51052, 0.51275, 0.49298)


In [22]:
info_file = os.path.join("output_%.5d" % dump, "info_%.5d.txt" % dump)
ds = yt.load(info_file)


yt : [INFO     ] 2023-11-15 13:21:58,224 Parameters: current_time              = 4.006944779175115
yt : [INFO     ] 2023-11-15 13:21:58,225 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2023-11-15 13:21:58,225 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-11-15 13:21:58,226 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-11-15 13:21:58,226 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-11-15 13:21:58,227 Parameters: current_redshift          = 8.999949195095816
yt : [INFO     ] 2023-11-15 13:21:58,227 Parameters: omega_lambda              = 0.723999977111816
yt : [INFO     ] 2023-11-15 13:21:58,227 Parameters: omega_matter              = 0.275999993085861
yt : [INFO     ] 2023-11-15 13:21:58,227 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-11-15 13:21:58,228 Parameters: hubble_constant           = 0.703000030517578


In [23]:
# define size and resolution of the region of interest

# small grid
amr_level = 13
box_size = 2 * const.kpc

# medium grid
#amr_level = 10
#box_size = 20 * const.kpc

# large grid
#amr_level = 7
#box_size = 100 * const.kpc

left_edge = halo_cat.coord[:, biggest_halo_idx] - box_size / 2
N = int(box_size / info.length_unit * ds.domain_dimensions[0] * 2**amr_level)
lowres = 2**(info.amr_level_max - amr_level)

print("Grid size: %d" % N)
print("Lowres factor: %d" % lowres)


Grid size: 147
Lowres factor: 1


In [24]:
# get raw yt data from the region of interest

smooth_grid = True

if smooth_grid:
    data = ds.smoothed_covering_grid(level=amr_level, left_edge=(left_edge / info.length_unit), dims=[N]*3)
else:
    data = ds.covering_grid(level=amr_level, left_edge=(left_edge / info.length_unit), dims=[N]*3)


In [25]:
# read particle data from yt

particle_type = data["io", "particle_family"].value.astype(int)
is_dm = particle_type == DM
is_star = particle_type == STAR

coord_dm = (np.array([data["io", "particle_position_x"].value[is_dm], data["io", "particle_position_y"].value[is_dm], data["io", "particle_position_z"].value[is_dm]])) * info.length_unit - left_edge[:, None] - box_size / 2
mass_dm = data["io", "particle_mass"][is_dm] * info.mass_unit

coord_star = (np.array([data["io", "particle_position_x"].value[is_star], data["io", "particle_position_y"].value[is_star], data["io", "particle_position_z"].value[is_star]])) * info.length_unit - left_edge[:, None] - box_size / 2
mass_star = data["io", "particle_mass"][is_star] * info.mass_unit
tau_starbirth = data["io", "conformal_birth_time"][is_star].value
id_star = data["io", "particle_identity"][is_star].value.astype(int)


yt : [WARNING  ] 2023-11-15 13:22:34,633 The DM particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,634 The star particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,635 The cloud particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,635 The dust particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,635 The star_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,635 The cloud_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,636 The dust_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-11-15 13:22:34,636 The gas_tracer particle filter already exists. Overriding.
yt : [INFO     ] 2023-11-15 13:22:34,636 Adding particle_type: DM
yt : [INFO     ] 2023-11-15 13:22:34,646 Adding particle_type: star
yt : [INFO     ] 2023-11-15 13:22:34,655 Adding particle_type: cloud
yt : [INFO     ] 2

In [26]:
# read gas data from yt

coord = (np.array([data["index", "x"].value, data["index", "y"].value, data["index", "z"].value])) * info.length_unit - left_edge[:, None, None, None] - box_size / 2
density = data["ramses", "Density"].value * info.density_unit
metallicity = data["ramses", "Metallicity"].value 
pressure = data["ramses", "Pressure"].value * info.energy_density_unit
energy_turb = data["ramses", "hydro_scalar_01"].value * info.vel_unit**2
ref_crit = data["ramses", "hydro_scalar_02"].value
vel_vec = np.array([data["ramses", "x-velocity"].value, data["ramses", "y-velocity"].value, data["ramses", "z-velocity"].value]) * info.vel_unit


In [27]:
grid_name = "data_d%d_l%d_b%d" % (dump, lowres, int(box_size / const.kpc))
print("filename: %s.npz" % grid_name)

filename: data_d41_l1_b2.npz


In [28]:
safe_savez(
    grid_name,
    halo_idx=halo_cat.idx[biggest_halo_idx],
    halo_mass=halo_cat.mass[biggest_halo_idx],
    box_size=box_size,
    left_edge=left_edge,
    N=N,
    coord_dm=coord_dm,
    mass_dm=mass_dm,
    coord_star=coord_star,
    mass_star=mass_star,
    tau_starbirth=tau_starbirth,
    id_star=id_star,
    coord=coord,
    density=density,
    metallicity=metallicity,
    pressure=pressure,
    energy_turb=energy_turb,
    ref_crit=ref_crit,
    vel_vec=vel_vec,
    lowres=lowres,
    **info.__dict__
)
